# Data Loading, Storage, and File Formats

The tools in this book are of little use if you can’t easily import and export data in
Python. I’m going to be focused on input and output with pandas objects, though there
are of course numerous tools in other libraries to aid in this process. NumPy, for example,
features low-level but extremely fast binary data loading and storage, including
support for memory-mapped array. See Chapter 12 for more on those.
Input and output typically falls into a few main categories: reading text files and other
more efficient on-disk formats, loading data from databases, and interacting with network
sources like web APIs.

## Reading and Writing Data in Text Format


Python has become a beloved language for text and file munging due to its simple syntax
for interacting with files, intuitive data structures, and convenient features like tuple
packing and unpacking.


pandas features a number of functions for reading tabular data as a DataFrame object.
Table 6-1 has a summary of all of them, though read_csv and read_table are likely the
ones you’ll use the most.


Table 6-1. Parsing functions in pandas

Function Description

read_csv Load delimited data from a file, URL, or file-like object. Use comma as default delimiter

read_table Load delimited data from a file, URL, or file-like object. Use tab ('\t') as default delimiter

read_fwf Read data in fixed-width column format (that is, no delimiters)

read_clipboard Version of read_table that reads data from the clipboard. Useful for converting tables from web pages

I’ll give an overview of the mechanics of these functions, which are meant to convert
text data into a DataFrame. The options for these functions fall into a few categories:

* Indexing: can treat one or more columns as the returned DataFrame, and whether to get column names from the file, the user, or not at all.

* Type inference and data conversion: this includes the user-defined value conversions and custom list of missing value markers.

* Datetime parsing: includes combining capability, including combining date and time information spread over multiple columns into a single column in the result.

* Iterating: support for iterating over chunks of very large files.

* Unclean data issues: skipping rows or a footer, comments, or other minor things like numeric data with thousands separated by commas.

Type inference is one of the more important features of these functions; that means you
don’t have to specify which columns are numeric, integer, boolean, or string. Handling
dates and other custom types requires a bit more effort, though. Let’s start with a small
comma-separated (CSV) text file:


In [3]:
!cat ex1.csv

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

Since this is comma-delimited, we can use read_csv to read it into a DataFrame:

In [97]:
from pandas import DataFrame, Series

import pandas as pd

import sys

import numpy as np

import json

In [7]:
df = pd.read_csv('ex1.csv')

In [8]:
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


We could also have used read_table and specifying the delimiter:

In [12]:
pd.read_table('ex1.csv',  sep=',')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


A file will not always have a header row. Consider this file:

In [13]:
!cat ex2.csv

1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

To read this in, you have a couple of options. You can allow pandas to assign default
column names, or you can specify names yourself:

In [14]:
pd.read_csv('ex2.csv', header=None)

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [15]:
pd.read_csv('ex2.csv', names=['a', 'b', 'c', 'd', 'message'])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


Suppose you wanted the message column to be the index of the returned DataFrame.
You can either indicate you want the column at index 4 or named 'message' using the
index_col argument:

In [16]:
names = ['a', 'b', 'c', 'd', 'message']

In [17]:
pd.read_csv('ex2.csv', names=names, index_col='message')

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


In the event that you want to form a hierarchical index from multiple columns, just
pass a list of column numbers or names:

In [18]:
!cat csv_mindex.csv

key1,key2,value1,value2
one,a,1,2
one,b,3,4
one,c,5,6
one,d,7,8
two,a,9,10
two,b,11,12
two,c,13,14
two,d,15,16


In [19]:
parsed = pd.read_csv('csv_mindex.csv', index_col=['key1', 'key2'])

In [20]:
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

In some cases, a table might not have a fixed delimiter, using whitespace or some other
pattern to separate fields. In these cases, you can pass a regular expression as a delimiter
for read_table. Consider a text file that looks like this:

In [21]:
list(open('ex3.csv'))

['            A         B         C\n',
 'aaa -0.264438 -1.026059 -0.619500\n',
 'bbb  0.927272  0.302904 -0.032399\n',
 'ccc -0.264273 -0.386314 -0.217601\n',
 'ddd -0.871858 -0.348382  1.100491']

While you could do some munging by hand, in this case fields are separated by a variable
amount of whitespace. This can be expressed by the regular expression \s+, so we have
then:

In [22]:
result = pd.read_table('ex3.txt', sep='\s+')

In [23]:
result

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


Because there was one fewer column name than the number of data rows, read_table
infers that the first column should be the DataFrame’s index in this special case.

The parser functions have many additional arguments to help you handle the wide
variety of exception file formats that occur (see Table 6-2). For example, you can skip
the first, third, and fourth rows of a file with skiprows:

In [24]:
!cat ex4.csv

# hey!
a,b,c,d,message
# just wanted to make things more difficult for you
# who reads CSV files with computers, anyway?
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [25]:
pd.read_csv('ex4.csv', skiprows=[0, 2, 3])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


Handling missing values is an important and frequently nuanced part of the file parsing
process. Missing data is usually either not present (empty string) or marked by some
sentinel value. By default, pandas uses a set of commonly occurring sentinels, such as
NA, -1.#IND, and NULL:

In [26]:
!cat ex5.csv

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo

In [27]:
result = pd.read_csv('ex5.csv')

In [28]:
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [29]:
pd.isnull(result)

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


The na_values option can take either a list or set of strings to consider missing values:

In [30]:
result = pd.read_csv('ex5.csv', na_values=['NULL'])

In [31]:
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


Different NA sentinels can be specified for each column in a dict:

In [33]:
sentinels = {'message': ['foo', 'NA'], 'something':['two']}

In [34]:
pd.read_csv('ex5.csv', na_values=sentinels)

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


Table 6-2. read_csv /read_table function arguments

Argument Description

path String indicating filesystem location, URL, or file-like object

sep or delimiter Character sequence or regular expression to use to split fields in each row

header Row number to use as column names. Defaults to 0 (first row), but should be None if there is no header row

index_col Column numbers or names to use as the row index in the result. Can be a single name/number or a list
of them for a hierarchical index

names List of column names for result, combine with header=None

skiprows Number of rows at beginning of file to ignore or list of row numbers (starting from 0) to skip

na_values Sequence of values to replace with NA

comment Character or characters to split comments off the end of lines

parse_dates Attempt to parse data to datetime; False by default. If True, will attempt to parse all columns. Otherwise
can specify a list of column numbers or name to parse. If element of list is tuple or list, will combine
multiple columns together and parse to date (for example if date/time split across two columns)

keep_date_col If joining columns to parse date, drop the joined columns. Default True

converters Dict containing column number of name mapping to functions. For example {'foo': f} would apply
the function f to all values in the 'foo' column

dayfirst When parsing potentially ambiguous dates, treat as international format (e.g. 7/6/2012 -> June 7,
2012). Default False

date_parser Function to use to parse dates

nrows Number of rows to read from beginning of file

iterator Return a TextParser object for reading file piecemeal

chunksize For iteration, size of file chunks

skip_footer Number of lines to ignore at end of file

verbose Print various parser output information, like the number of 
missing values placed in non-numeric
columns

encoding Text encoding for unicode. For example 'utf-8' for UTF-8 encoded text

squeeze If the parsed data only contains one column return a Series

thousands Separator for thousands, e.g. ',' or '.'

## Reading Text Files in Pieces


When processing very large files or figuring out the right set of arguments to correctly
process a large file, you may only want to read in a small piece of a file or iterate through
smaller chunks of the file.

In [35]:
result = pd.read_csv('ex6.csv')

In [36]:
result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
5,1.817480,0.742273,0.419395,-2.251035,Q
6,-0.776764,0.935518,-0.332872,-1.875641,U
7,-0.913135,1.530624,-0.572657,0.477252,K
8,0.358480,-0.497572,-0.367016,0.507702,S
9,-1.740877,-1.160417,-1.637830,2.172201,G


If you want to only read out a small number of rows (avoiding reading the entire file),
specify that with nrows:

In [37]:
pd.read_csv('ex6.csv', nrows=5)

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


To read out a file in pieces, specify a chunksize as a number of rows:

In [38]:
chunker = pd.read_csv('ex6.csv', chunksize=1000)

In [39]:
chunker

The TextParser object returned by read_csv allows you to iterate over the parts of the
file according to the chunksize. For example, we can iterate over ex6.csv, aggregating
the value counts in the 'key' column like so:

In [48]:
chunker = pd.read_csv('ex6.csv', chunksize=1000)

In [49]:
tot = Series([])
for piece in chunker:
    tot = tot.add(piece['key'].value_counts(), fill_value=0)
    
tot = tot.sort_values(ascending=False)

We have then:


In [50]:
tot[:10]

E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
M    338.0
J    337.0
F    335.0
K    334.0
H    330.0
dtype: float64

In [47]:
tot

Series([], dtype: float64)

TextParser is also equipped with a get_chunk method which enables you to read pieces
of an arbitrary size.

## Writing Data Out to Text Format

Data can also be exported to delimited format. Let’s consider one of the CSV files read
above:

In [51]:
data = pd.read_csv('ex5.csv')

In [53]:
data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


Using DataFrame’s to_csv method, we can write the data out to a comma-separated file:

In [54]:
data.to_csv('out.csv')

In [55]:
!cat out.csv

,something,a,b,c,d,message
0,one,1,2,3.0,4,
1,two,5,6,,8,world
2,three,9,10,11.0,12,foo


Other delimiters can be used, of course (writing to sys.stdout so it just prints the text
result):

In [59]:
data.to_csv(sys.stdout, sep='|')

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


Missing values appear as empty strings in the output. You might want to denote them
by some other sentinel value:

In [60]:
data.to_csv(sys.stdout, na_rep='NULL')

,something,a,b,c,d,message
0,one,1,2,3.0,4,NULL
1,two,5,6,NULL,8,world
2,three,9,10,11.0,12,foo


With no other options specified, both the row and column labels are written. Both of
these can be disabled:

In [61]:
data.to_csv(sys.stdout, index=False, header=False)

one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo


You can also write only a subset of the columns, and in an order of your choosing:

In [63]:
data.to_csv(sys.stdout, index=False, columns=['a', 'b', 'c'])

a,b,c
1,2,3.0
5,6,
9,10,11.0


Series also has a to_csv method:

In [65]:
dates = pd.date_range('1/1/2000', periods=7)

In [68]:
ts = Series(np.arange(7), index=dates)

In [70]:
ts.to_csv('tseries.csv')

In [71]:
!cat tseries.csv

2000-01-01,0
2000-01-02,1
2000-01-03,2
2000-01-04,3
2000-01-05,4
2000-01-06,5
2000-01-07,6


With a bit of wrangling (no header, first column as index), you can read a CSV version
of a Series with read_csv, but there is also a from_csv convenience method that makes
it a bit simpler:

In [72]:
Series.from_csv('tseries.csv', parse_dates=True)

2000-01-01    0
2000-01-02    1
2000-01-03    2
2000-01-04    3
2000-01-05    4
2000-01-06    5
2000-01-07    6
dtype: int64

See the docstrings for to_csv and from_csv in IPython for more information.

## Manually Working with Delimited Formats

Most forms of tabular data can be loaded from disk using functions like pan
das.read_table. In some cases, however, some manual processing may be necessary.
It’s not uncommon to receive a file with one or more malformed lines that trip up
read_table. To illustrate the basic tools, consider a small CSV file:

In [73]:
!cat ex7.csv

"a","b","c"
"1","2","3"
"1","2","3","4"


For any file with a single-character delimiter, you can use Python’s built-in csv module.
To use it, pass any open file or file-like object to csv.reader:

In [76]:
import csv
f = open('ex7.csv')

reader = csv.reader(f)

Iterating through the reader like a file yields tuples of values in each like with any quote
characters removed:

In [78]:
for line in reader:
    print (line)

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3', '4']


From there, it’s up to you to do the wrangling necessary to put the data in the form
that you need it. For example:

In [79]:
lines = list(csv.reader(open('ex7.csv')))

In [80]:
header, values = lines[0], lines[1:]

In [81]:
data_dict = {h: v for h, v in zip(header, zip(*values))}

In [82]:
data_dict

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

CSV files come in many different flavors. Defining a new format with a different delimiter,
string quoting convention, or line terminator is done by defining a simple subclass
of csv.Dialect:

In [85]:
# class my_dialect(csv.Dialect):
#     lineterminator = '\n'
#     delimiter = ';'
#     quotechar = '"'

class my_dialect(csv.Dialect):
    lineterminator = '\n'
    delimiter = ';'
    quotechar = '"'

In [88]:
reader = csv.reader(f, dialect=my_dialect, quoting=csv.QUOTE_NONE)
reader

Individual CSV dialect parameters can also be given as keywords to csv.reader without
having to define a subclass:

In [89]:
reader = csv.reader(f,delimiter='|')

The possible options (attributes of csv.Dialect) and what they do can be found in

Table 6-3.

Table 6-3. CSV dialect options

Argument Description

delimiter One-character string to separate fields. Defaults to ','.

lineterminator Line terminator for writing, defaults to '\r\n'. Reader ignores this and recognizes

cross-platform line terminators.

quotechar Quote character for fields with special characters (like a delimiter). Default is '"'.

quoting Quoting convention. Options include csv.QUOTE_ALL (quote all fields), csv.QUOTE_MINIMAL (only fields with special characters like the delimiter),csv.QUOTE_NONNUMERIC, and csv.QUOTE_NON (no quoting). See Python’s documentation for full details. Defaults to QUOTE_MINIMAL.

skipinitialspace Ignore whitespace after each delimiter. Default False.

doublequote How to handle quoting character inside a field. If True, it is doubled. See online
documentation for full detail and behavior.

escapechar String to escape the delimiter if quoting is set to csv.QUOTE_NONE. Disabled by
default

For files with more complicated or fixed multicharacter delimiters, you
will not be able to use the csv module. In those cases, you’ll have to do
the line splitting and other cleanup using string’s split method or the
regular expression method re.split.

To write delimited files manually, you can use csv.writer. It accepts an open, writable
file object and the same dialect and format options as csv.reader:

In [92]:
with open('mydata.csv', 'w') as f:
    writer = csv.writer(f, dialect=my_dialect, quoting=csv.QUOTE_NONE)
    writer.writerow(('one', 'two', 'three'))
    writer.writerow(('1', '2', '3'))
    writer.writerow(('4', '5', '6'))
    writer.writerow(('7', '8', '9'))

## JSON Data

JSON (short for JavaScript Object Notation) has become one of the standard formats
for sending data by HTTP request between web browsers and other applications. It is
a much more flexible data format than a tabular text form like CSV. Here is an example:

In [93]:
obj = """
{"name": "Wes",
"places_lived": ["United States", "Spain", "Germany"],
"pet": null,
"siblings": [{"name": "Scott", "age": 25, "pet": "Zuko"},
{"name": "Katie", "age": 33, "pet": "Cisco"}]
}
"""

In [94]:
obj

'\n{"name": "Wes",\n"places_lived": ["United States", "Spain", "Germany"],\n"pet": null,\n"siblings": [{"name": "Scott", "age": 25, "pet": "Zuko"},\n{"name": "Katie", "age": 33, "pet": "Cisco"}]\n}\n'

JSON is very nearly valid Python code with the exception of its null value null and
some other nuances (such as disallowing trailing commas at the end of lists). The basic
types are objects (dicts), arrays (lists), strings, numbers, booleans, and nulls. All of the
keys in an object must be strings. There are several Python libraries for reading and
writing JSON data. I’ll use json here as it is built into the Python standard library. To
convert a JSON string to Python form, use json.loads:

In [96]:
import json

In [98]:
result = json.loads(obj)

In [99]:
result

{'name': 'Wes',
 'pet': None,
 'places_lived': ['United States', 'Spain', 'Germany'],
 'siblings': [{'age': 25, 'name': 'Scott', 'pet': 'Zuko'},
  {'age': 33, 'name': 'Katie', 'pet': 'Cisco'}]}

json.dumps on the other hand converts a Python object back to JSON:

In [100]:
asjson = json.dumps(result)

In [101]:
asjson

'{"pet": null, "name": "Wes", "places_lived": ["United States", "Spain", "Germany"], "siblings": [{"pet": "Zuko", "name": "Scott", "age": 25}, {"pet": "Cisco", "name": "Katie", "age": 33}]}'

How you convert a JSON object or list of objects to a DataFrame or some other data
structure for analysis will be up to you. Conveniently, you can pass a list of JSON objects
to the DataFrame constructor and select a subset of the data fields:

In [102]:
siblings = DataFrame(result['siblings'], columns=['name', 'age'])

In [103]:
siblings

,name,age
0,Scott,25
1,Katie,33


In [104]:
siblings_test = DataFrame(result, columns=['pet', 'name', 'places_lived', 'siblings'])

ValueError: arrays must all be same length

For an extended example of reading and manipulating JSON data (including nested
records), see the USDA Food Database example in the next chapter.

NOTE
An effort is underway to add fast native JSON export (to_json) and
decoding (from_json) to pandas. This was not ready at the time of writing.

## XML and HTML: Web Scraping

Python has many libraries for reading and writing data in the ubiquitous HTML and
XML formats. lxml (http://lxml.de) is one that has consistently strong performance in
parsing very large files. lxml has multiple programmer interfaces; first I’ll show using
lxml.html for HTML, then parse some XML using lxml.objectify.

Many websites make data available in HTML tables for viewing in a browser, but not
downloadable as an easily machine-readable format like JSON, HTML, or XML. I noticed
that this was the case with Yahoo! Finance’s stock options data. If you aren’t
familiar with this data; options are derivative contracts giving you the right to buy
(call option) or sell (put option) a company’s stock at some particular price (the
strike) between now and some fixed point in the future (the expiry). People trade both
call and put options across many strikes and expiries; this data can all be found together
in tables on Yahoo! Finance.

To get started, find the URL you want to extract data from, open it with urllib2 and
parse the stream with lxml like so:

In [108]:
from lxml.html import parse
# from urllib2 import urlopen
from urllib.request import urlopen

parsed = parse(urlopen('http://finance.yahoo.com/q/op?s=AAPL+Options'))

doc = parsed.getroot()

Using this object, you can extract all HTML tags of a particular type, such as table tags
containing the data of interest. As a simple motivating example, suppose you wanted
to get a list of every URL linked to in the document; links are a tags in HTML. Using
the document root’s findall method along with an XPath (a means of expressing
“queries” on the document):

In [109]:
links = doc.findall('.//a')

In [112]:
links[15:20]

[<Element a at 0x10f14d598>,
 <Element a at 0x10f14d5e8>,
 <Element a at 0x10f14d638>,
 <Element a at 0x10f14d688>,
 <Element a at 0x10f14d6d8>]

But these are objects representing HTML elements; to get the URL and link text you
have to use each element’s get method (for the URL) and text_content method (for
the display text):

In [113]:
lnk = links[28]

In [114]:
lnk

<Element a at 0x10f14d9a8>

In [115]:
lnk.get('href')

'/quote/AMZN?p=AMZN'

In [116]:
lnk.text_content()

'AMZN'

Thus, getting a list of all URLs in the document is a matter of writing this list comprehension:

In [117]:
urls = [lnk.get('href') for lnk in doc.findall('.//a')]

In [119]:
urls[-10:]

['/quote/AAPL/analysts?p=AAPL',
 'https://help.yahoo.com/kb/index?page=content&y=PROD_MAIL_ML&locale=en_US&id=SLN2310&actp=productlink',
 'http://help.yahoo.com/l/us/yahoo/finance/',
 'https://yahoo.uservoice.com/forums/382977',
 'http://info.yahoo.com/privacy/us/yahoo/',
 'http://info.yahoo.com/relevantads/',
 'http://info.yahoo.com/legal/us/yahoo/utos/utos-173.html',
 'http://twitter.com/YahooFinance',
 'http://facebook.com/yahoofinance',
 'http://yahoofinance.tumblr.com']

Now, finding the right tables in the document can be a matter of trial and error; some
websites make it easier by giving a table of interest an id attribute. I determined that
these were the two tables containing the call data and put data, respectively:

In [121]:
tables = doc.findall('.//table')

# calls = tables[9]
# puts = tables[13]

In [124]:
tables[0]

<Element table at 0x10f15c048>

Each table has a header row followed by each of the data rows:

In [125]:
# rows = calls.findall('.//tr')

For the header as well as the data rows, we want to extract the text from each cell; in
the case of the header these are th cells and td cells for the data:

In [126]:
# def _unpack(row, kind='td'):
#     elts = row.findall('.//%s' % kind)
#     return [val.text_content() for val in elts]

Thus, we obtain:

In [127]:
# _unpack(rows[0], kind='th')

In [128]:
# _unpack(rows[1], kind='td')

Now, it’s a matter of combining all of these steps together to convert this data into a
DataFrame. Since the numerical data is still in string format, we want to convert some,
but perhaps not all of the columns to floating point format. You could do this by hand,
but, luckily, pandas has a class TextParser that is used internally in the read_csv and
other parsing functions to do the appropriate automatic type conversion:

In [129]:
from pandas.io.parsers import TextParser

In [130]:
def parse_options_data(table):
    rows = table.findall('.//tr')
    header = _unpack(rows[0], kind='th')
    data = [_unpack(r) for r in rows[1:]]
    return TextParser(data, names=header).get_chunk()

Finally, we invoke this parsing function on the lxml table objects and get DataFrame
results:

In [132]:
# call_data = parse_options_data(calls)

NameError: name 'calls' is not defined

In [133]:
# put_data = parse_options_data(puts)

In [134]:
# call_data[:10]

Parsing XML with lxml.objectify

XML (extensible markup language) is another common structured data format supporting
hierarchical, nested data with metadata. The files that generate the book you
are reading actually form a series of large XML documents.

Above, I showed the lxml library and its lxml.html interface. Here I show an alternate
interface that’s convenient for XML data, lxml.objectify.

The New York Metropolitan Transportation Authority (MTA) publishes a number of
data series about its bus and train services (http://www.mta.info/developers/download
.html). Here we’ll look at the performance data which is contained in a set of XML files.
Each train or bus service has a different file (like Performance_MNR.xml for the Metro-
North Railroad) containing monthly data as a series of XML records that look like this:

<INDICATOR>
<INDICATOR_SEQ>373889</INDICATOR_SEQ>
<PARENT_SEQ></PARENT_SEQ>
<AGENCY_NAME>Metro-North Railroad</AGENCY_NAME>
<INDICATOR_NAME>Escalator Availability</INDICATOR_NAME>
<DESCRIPTION>Percent of the time that escalators are operational
systemwide. The availability rate is based on physical observations performed
the morning of regular business days only. This is a new indicator the agency
began reporting in 2009.</DESCRIPTION>
<PERIOD_YEAR>2011</PERIOD_YEAR>
<PERIOD_MONTH>12</PERIOD_MONTH>
<CATEGORY>Service Indicators</CATEGORY>
<FREQUENCY>M</FREQUENCY>
<DESIRED_CHANGE>U</DESIRED_CHANGE>
<INDICATOR_UNIT>%</INDICATOR_UNIT>
<DECIMAL_PLACES>1</DECIMAL_PLACES>
<YTD_TARGET>97.00</YTD_TARGET>
<YTD_ACTUAL></YTD_ACTUAL>
<MONTHLY_TARGET>97.00</MONTHLY_TARGET>
<MONTHLY_ACTUAL></MONTHLY_ACTUAL>
</INDICATOR>

Using lxml.objectify, we parse the file and get a reference to the root node of the XML
file with getroot:

In [137]:
from lxml import objectify

path = 'Performance_MNR.xml'
parsed = objectify.parse(open(path))
root = parsed.getroot()

root.INDICATOR return a generator yielding each <INDICATOR> XML element. For each
record, we can populate a dict of tag names (like YTD_ACTUAL) to data values (excluding
a few tags):

In [139]:
data = []

skip_fields = ['PARENT_SEQ', 'INDICATOR_SEQ', 'DESIRED_CHANGE', 'DECIMAL_PLACES']

for elt in root.INDICATOR:
    el_data = {}
    for child in elt.getchildren():
        if child.tag in skip_fields:
            continue
        el_data[child.tag] = child.pyval
    data.append(el_data)

Lastly, convert this list of dicts into a DataFrame:

In [140]:
perf = DataFrame(data)

In [141]:
perf

,AGENCY_NAME,CATEGORY,DESCRIPTION,FREQUENCY,INDICATOR_NAME,INDICATOR_UNIT,MONTHLY_ACTUAL,MONTHLY_TARGET,PERIOD_MONTH,PERIOD_YEAR,YTD_ACTUAL,YTD_TARGET
0,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96.9,95,1,2008,96.9,95
1,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,95,95,2,2008,96,95
2,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96.9,95,3,2008,96.3,95
3,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,98.3,95,4,2008,96.8,95
4,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,95.8,95,5,2008,96.6,95
5,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,94.4,95,6,2008,96.2,95
6,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96,95,7,2008,96.2,95
7,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96.4,95,8,2008,96.2,95
8,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,93.7,95,9,2008,95.9,95
9,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96.4,95,10,2008,96,95


XML data can get much more complicated than this example. Each tag can have metadata,
too. Consider an HTML link tag which is also valid XML:

In [144]:
#from StringIO import StringIO
# The StringIO and cStringIO modules are gone. 
#Instead, import the io module and use io.StringIO or io.BytesIO 
#for text and data respectively.

from io import StringIO
tag = '<a href="http://www.google.com">Google</a>'

root = objectify.parse(StringIO(tag)).getroot()

You can now access any of the fields (like href) in the tag or the link text:

In [145]:
root

<Element a at 0x10f56e208>

In [146]:
root.get('href')

'http://www.google.com'

In [147]:
root.text

'Google'